In [15]:
import warnings
import pandas as pd

from datetime import datetime

In [16]:
def parseData(row):
    return datetime.strptime(row['Data'], "%Y-%m-%d")

def getCity(city, cities):
    dataframe = cities[cities['City'] == city]
    dataframe = dataframe[['16-06', '22-06', '29-06', '06-07', '13-07', '20-07', '10-08', '18-08', '24-08', '01-09', '07-09', '15-09', '21-09']]
    return dataframe.sum()

def groupCovid(df, start, end):
    df = df.sort_values(by="Data")
    df = df[(df['Data'] >= start) & (df['Data'] <= end)]
    
    groupedByWeek = df.groupby(df['Data'].dt.strftime('%W'))['casos']
    
    return groupedByWeek.sum()

def analyzeCity(city, covid_dataset) -> pd.DataFrame:
    #remover colunas unamed
    covid_dataset.drop(covid_dataset.columns[covid_dataset.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
    # covid_dataset['Data'] = covid_dataset.apply (lambda row: parseData(row), axis=1)
    covid_dataset['Data'] = pd.to_datetime(covid_dataset['Data'])
    df = covid_dataset[covid_dataset['cidade'] == city]

    tweets_per_city = pd.read_csv('../data/distribuicao-tweets/distribuitionWithCity.csv')

    tweetsNormalizados = getCity(city, tweets_per_city)
    better_correlation = -1
    better_weeks_after = 0
    melhor_df = pd.DataFrame()

    #normaliza dados
    tweetsNormalizados=(tweetsNormalizados-tweetsNormalizados.min())/(tweetsNormalizados.max()-tweetsNormalizados.min())
    covidDataByWeek = groupCovid(df, "2020-06-30", "2020-09-21")

    #normaliza dados
    covidDataByWeek=(covidDataByWeek-covidDataByWeek.min())/(covidDataByWeek.max()-covidDataByWeek.min())
    dfTweets = pd.DataFrame({'data':tweetsNormalizados.index, 'tweets na semana':tweetsNormalizados.values})
    dfTweets["id"] = dfTweets.index

    dfCasos = pd.DataFrame({'semana':covidDataByWeek.index, 'casos de covid na semana':covidDataByWeek.values})

    for weeksAfter in range(0, 5):
        dfCasos["id"] = dfCasos.index + weeksAfter #compara com casos X semanas seguintes

        dadosAgrupados = pd.concat([dfTweets])
        dadosAgrupados = pd.merge(dadosAgrupados, dfCasos, on='id')
        dadosAgrupados = dadosAgrupados[['data', 'semana', 'tweets na semana', 'casos de covid na semana']]
        current_correlation = dadosAgrupados['tweets na semana'].corr(dadosAgrupados['casos de covid na semana'])

        # assume correlações negativas como boas
        # current_correlation = abs(current_correlation)
        if current_correlation > better_correlation and current_correlation > 0.4:
            better_correlation = current_correlation
            better_weeks_after = weeksAfter
            melhor_df = dadosAgrupados

    return better_correlation, better_weeks_after, melhor_df

df = pd.read_csv('../data/casos/covid-cases-full.csv')
cities = df['cidade'].unique()
#cities = ["São Paulo", "Rio de Janeiro", "Belo Horizonte", "Fortaleza", "Porto Alegre", "Recife", "Manaus", "Salvador"]

dataframes = [
    pd.DataFrame(columns=['data', 'semana', 'tweets na semana', 'casos de covid na semana', 'cidade']),
    pd.DataFrame(columns=['data', 'semana', 'tweets na semana', 'casos de covid na semana', 'cidade']),
    pd.DataFrame(columns=['data', 'semana', 'tweets na semana', 'casos de covid na semana', 'cidade']),
    pd.DataFrame(columns=['data', 'semana', 'tweets na semana', 'casos de covid na semana', 'cidade']),
    pd.DataFrame(columns=['data', 'semana', 'tweets na semana', 'casos de covid na semana', 'cidade'])
]

for city in cities:
    correlation, weeks_after, melhor_df = analyzeCity(city, df)
    print(city, correlation, weeks_after)
    if correlation > -1:
        melhor_df['cidade'] = city
        dataframes[weeks_after] = pd.concat([dataframes[weeks_after], melhor_df])

weeks = 0
for df in dataframes:
    df.to_csv(f'../data/balaios/grupo_{weeks}.csv', index=False)
    weeks += 1

São Paulo 0.6926187275688374 4
Barra Mansa -1 0
Feira de Santana 0.706773477404403 2
Rio de Janeiro 0.43503843885066695 0
Brasília -1 0
Santana de Parnaíba -1 0
Divinópolis 0.4476942728507031 4
Maceió 0.7432913524900342 4
Campo Bom -1 0
Porto Alegre -1 0
Caxias do Sul -1 0
Cianorte -1 0
Curitiba 0.5266745043705867 0
Ferraz de Vasconcelos -1 0
Florianópolis -1 0
Goiânia 0.43118793021132895 3
Ipatinga -1 0
Natal 0.5816625538752715 1
Niterói -1 0
Recife 0.5794163360957665 3
Rio Verde -1 0
Joinville -1 0
Manaus -1 0
Salvador -1 0
Aracaju 0.4850137575345954 4
Campo Grande -1 0
Carapicuíba -1 0
Importados/Indefinidos -1 0
Juiz de Fora -1 0
Patrocínio -1 0
Rancho Queimado -1 0
Braço do Norte -1 0
Anápolis 0.5217883407183515 3
Aquiraz -1 0
Belo Horizonte 0.6178744387728696 0
Fortaleza 0.6318967932475601 2
Mauá -1 0
Porto Seguro -1 0
Santo André 0.4381897203560213 2
São Bernardo do Campo 0.540077146392207 1
São Caetano do Sul -1 0
Belo Jardim -1 0
Campo Largo -1 0
Coronel Fabriciano 0.568606305